# 03 - Interactive Viz

## Deadline

Wednesday November 8th, 2017 at 11:59PM

## Important Notes

- Make sure you push on GitHub your Notebook with all the cells already evaluated
- Note that maps do not render in a standard Github environment : you should export them to HTML and link them in your notebook.
- Remember that `.csv` is not the only data format. Though they might require additional processing, some formats provide better encoding support.
- Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you plan to implement!
- Please write all your comments in English, and use meaningful variable names in your code

## Background

In this homework we will be exploring interactive visualization, which is a key ingredient of many successful data visualizations (especially when it comes to infographics).

Unemployment rates are major economic metrics and a matter of concern for governments around the world. Though its definition may seem straightforward at first glance (usually defined as the number of unemployed people divided by the active population), it can be tricky to define consistently. For example, one must define what exactly unemployed means : looking for a job ? Having declared their unemployment ? Currently without a job ? Should students or recent graduates be included ? We could also wonder what the active population is : everyone in an age category (e.g. `16-64`) ? Anyone interested by finding a job ? Though these questions may seem subtle, they can have a large impact on the interpretation of the results : `3%` unemployment doesn't mean much if we don't know who is included in this percentage. 

In this homework you will be dealing with two different datasets from the statistics offices of the European commission ([eurostat](http://ec.europa.eu/eurostat/data/database)) and the Swiss Confederation ([amstat](https://www.amstat.ch)). They provide a variety of datasets with plenty of information on many different statistics and demographics at their respective scales. Unfortunately, as is often the case is data analysis, these websites are not always straightforward to navigate. They may include a lot of obscure categories, not always be translated into your native language, have strange link structures, â€¦ Navigating this complexity is part of a data scientists' job : you will have to use a few tricks to get the right data for this homework.

For the visualization part, install [Folium](https://github.com/python-visualization/folium) (*HINT*: it is not available in your standard Anaconda environment, therefore search on the Web how to install it easily!). Folium's `README` comes with very clear examples, and links to their own iPython Notebooks -- make good use of this information. For your own convenience, in this same directory you can already find two `.topojson` files, containing the geo-coordinates of 

- European countries (*liberal definition of EU*) (`topojson/europe.topojson.json`, [source](https://github.com/leakyMirror/map-of-europe))
- Swiss cantons (`topojson/ch-cantons.topojson.json`) 

These will be used as an overlay on the Folium maps.

## Assignment

1. Go to the [eurostat](http://ec.europa.eu/eurostat/data/database) website and try to find a dataset that includes the european unemployment rates at a recent date.

   Use this data to build a [Choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) which shows the unemployment rate in Europe at a country level. Think about [the colors you use](https://carto.com/academy/courses/intermediate-design/choose-colors-1/), how you decided to [split the intervals into data classes](http://gisgeography.com/choropleth-maps-data-classification/) or which interactions you could add in order to make the visualization intuitive and expressive. Compare Switzerland's unemployment rate to that of the rest of Europe.

2. Go to the [amstat](https://www.amstat.ch) website to find a dataset that includes the unemployment rates in Switzerland at a recent date.

   > *HINT* Go to the `details` tab to find the raw data you need. If you do not speak French, German or Italian, think of using free translation services to navigate your way through. 

   Use this data to build another Choropleth map, this time showing the unemployment rate at the level of swiss cantons. Again, try to make the map as expressive as possible, and comment on the trends you observe.

   The Swiss Confederation defines the rates you have just plotted as the number of people looking for a job divided by the size of the active population (scaled by 100). This is surely a valid choice, but as we discussed one could argue for a different categorization.

   Copy the map you have just created, but this time don't count in your statistics people who already have a job and are looking for a new one. How do your observations change ? You can repeat this with different choices of categories to see how selecting different metrics can lead to different interpretations of the same data.

3. Use the [amstat](https://www.amstat.ch) website again to find a dataset that includes the unemployment rates in Switzerland at recent date, this time making a distinction between *Swiss* and *foreign* workers.

   The Economic Secretary (SECO) releases [a monthly report](https://www.seco.admin.ch/seco/fr/home/Arbeit/Arbeitslosenversicherung/arbeitslosenzahlen.html) on the state of the employment market. In the latest report (September 2017), it is noted that there is a discrepancy between the unemployment rates for *foreign* (`5.1%`) and *Swiss* (`2.2%`) workers. 

   Show the difference in unemployment rates between the two categories in each canton on a Choropleth map (*hint* The easy way is to show two separate maps, but can you think of something better ?). Where are the differences most visible ? Why do you think that is ?

   Now let's refine the analysis by adding the differences between age groups. As you may have guessed it is nearly impossible to plot so many variables on a map. Make a bar plot, which is a better suited visualization tool for this type of multivariate data.

4. *BONUS*: using the map you have just built, and the geographical information contained in it, could you give a *rough estimate* of the difference in unemployment rates between the areas divided by the [RÃ¶stigraben](https://en.wikipedia.org/wiki/R%C3%B6stigraben)?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Web scraping
import requests
from bs4 import BeautifulSoup
import json

# Merging names. Installation: pip install fuzzywuzzy and pip install fuzzywuzzy[speedup]
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from IPython.core import display as ICD

import folium

In [101]:
JSON_FOLDER = r'topojson/'
country_geo_path = r'topojson/europe.topojson.json'
geo_json_data_europe = json.load(open(country_geo_path))  # load coordinates for european countries
geo_json_data_europe

{'arcs': [[[9439, 2245],
   [-14, 6],
   [-31, 9],
   [-31, 11],
   [-12, 5],
   [-11, 5],
   [-2, 7],
   [0, 8],
   [-10, 25],
   [-5, 9],
   [-2, 1],
   [-1, -2],
   [-3, -2],
   [-16, 7],
   [-1, 2],
   [1, 4],
   [-2, 11],
   [-2, 6],
   [-7, 14],
   [-9, 11],
   [-3, 1],
   [-3, 3],
   [-3, 11],
   [-3, 14],
   [-2, 17],
   [-2, 3],
   [-8, 2]],
  [[9257, 2433], [0, 5], [-3, 12], [-2, 2]],
  [[9252, 2452],
   [3, 2],
   [8, 3],
   [3, 0],
   [5, -2],
   [7, 4],
   [3, 4],
   [4, 6],
   [5, 1],
   [3, -3],
   [24, -37],
   [7, -17],
   [2, -2],
   [17, -11],
   [5, 2],
   [12, 12],
   [17, 8],
   [3, 0],
   [8, -5],
   [3, -3],
   [1, -27],
   [4, -21],
   [18, -18],
   [2, -6],
   [0, -2],
   [-5, -8],
   [0, -10],
   [18, -55],
   [10, -22]],
  [[9313, 2757], [-1, -2], [-3, 3], [-1, 3], [2, 2], [3, -4], [0, -2]],
  [[9288, 2771],
   [-3, -1],
   [-2, 1],
   [-3, 3],
   [-1, 3],
   [1, 5],
   [1, 2],
   [5, -1],
   [3, -5],
   [0, -5],
   [-1, -2]],
  [[9760, 2963],
   [4, -3],
  

In [18]:
canton_geo_path = JSON_FOLDER + r'ch-cantons.topojson.json'
geo_json_data_swiss = json.load(open(canton_geo_path))  # load coordinates for swiss cantons

In [53]:
m_europe = folium.Map([50,5], tiles='cartodbpositron', zoom_start=4)
m_europe

In [ ]:
## tuto 1 (no need to run)
ice_map = folium.Map(location=[-59.1759, -11.6016],
                   tiles='Mapbox Bright', zoom_start=2)

folium.GeoJson(open('antarctic_ice_edge.json'),
               name='geojson'
              ).add_to(ice_map)

folium.TopoJson(open('antarctic_ice_shelf_topo.json'),
                'objects.antarctic_ice_shelf',
                name='topojson',
               ).add_to(ice_map)

folium.LayerControl().add_to(ice_map)
ice_map

'objects': {'antarctic_ice_shelf': {'geometries': [

In [ ]:
## tuto2 (not ran)
geo_path = r'data/antarctic_ice_edge.json'
topo_path = r'data/antarctic_ice_shelf_topo.json'

ice_map = folium.Map(location=[-59.1759, -11.6016],
                     tiles='Mapbox Bright', zoom_start=2)
ice_map.geo_json(geo_path=geo_path)
ice_map.geo_json(geo_path=topo_path, topojson='objects.antarctic_ice_shelf')
ice_map.create_map()

In [68]:
# not working
## 'objects': {'europe
topo_path = r'topojson/europe.topojson.json'
eu_map = folium.Map(location = [50,5], tiles='cartodbpositron', zoom_start=4)
eu_map.json(geo_path=topo_path, topojson='id.DK')
eu_map.create_map()

AttributeError: 'Map' object has no attribute 'json'

In [80]:
topo_path = r'topojson/europe.topojson.json'
eu_map = folium.Map(location = [50,5], tiles='cartodbpositron', zoom_start=4)

folium.TopoJson(open(topo_path),
                'objects.europe',
                name='topojson',
               ).add_to(eu_map)

In [82]:
## use it if you want to ad more layers... here i does basically nothing i think :p
folium.LayerControl().add_to(eu_map)

In [83]:
eu_map

In [ ]:
# not to be ran
state_geo_path = r'us-states.json'
geo_json_data = json.load(open(state_geo_path))

def us_election_colors(state):
    """Maps Clinton wins to blue and Trump wins to red."""
    clinton_win = state_data.loc[state_data['State'] == state, 'Winner'].values[0] > 1
    if clinton_win:
        return '#0000ff'
    else:
        return '#ff0000'

In [52]:
# not to be ran just for momentany understanding and copy paste
test_map = folium.Map([50,5], tiles='cartodbpositron', zoom_start=4)
folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        'fillColor': us_election_colors(feature['id']),
        'color' : 'black',
        'weight' : 2,
        'dashArray' : '5, 5'
        }
    ).add_to(test_map)

test_map

In [87]:
test_map = folium.Map([50,5], tiles='Mapbox Control Room', zoom_start=4)
folium.GeoJson(
    geo_json_data_europe,
    style_function=lambda feature: {
        'fillColor': 0,
        'color' : 'black',
        'weight' : 2,
        'dashArray' : '5, 5'
        }
    ).add_to(test_map)

test_map

Data:  **Long-term unemployment rate, % of active population aged 15-74**       _from eurostat_

link: http://ec.europa.eu/eurostat/web/products-datasets/-/tipslm70

Short Description: The long-term unemployment rate is the number of persons unemployed for 12 months or longer as a percentage of the labour force (i.e. economically active population). The unemployment rate is the number of unemployed persons as a percentage of the labour force (the total number of people employed and unemployed) based on International Labour Office (ILO) definition. Unemployed persons comprise persons aged 15 to 74 who fulfil all the three following conditions: are without work during the reference week; are available to start work within the next two weeks and have been actively seeking work in the past four weeks or have already found a job to start within the next three months. The MIP Scoreboard indicator is the three years change in percentage points. The indicative threshold is 0.5 p..
The data source is the quarterly EU Labour Force Survey (EU LFS). The survey covers the resident population in private households. 

* Name of file: tipslm70.tsv
* **:** are non available data
* **b** break in the time serie
* **e** for estimated
* **u** low relialability
* **PC** for percentage
* **PPCH** for percentage point change t/(t-3)

where:      
_A percentage point or percent point (pp) is the unit for the arithmetic difference of two percentages. For example, moving up from 40% to 44% is a 4 percentage point increase, but is an actual 10 percent increase in what is being measured._


* For data by recent months, by sex: http://ec.europa.eu/eurostat/web/products-datasets/-/teilm020
* Recent months 25-74: http://ec.europa.eu/eurostat/web/products-datasets/-/teilm022

In [88]:
DATA_FOLDER = 'data/'
unemp_df1 = pd.read_csv(DATA_FOLDER + 'tipslm70.tsv' , sep='\t|,|[|]', engine='python')

#unemp_df1_PC = unemp_df1.loc[unemp_df1['unit,sex,geo\\time']=='PC_ACT,T,AT',:]
unemp_df1.head()
#unemp_df1.loc[unemp_df1['unit,sex,geo\\time']=='PC_ACT,T,AT',:]['unit,sex,geo\\time']

,unit,sex,geo\time,1996,1997,1998,1999,2000,2001,2002,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,PC_ACT,T,AT,:,:,:,:,:,:,1.1,...,1.3 b,1.0,1.2,1.2,1.2,1.2,1.3,1.5,1.7,1.9
1,PC_ACT,T,BE,:,:,:,4.9 b,3.8,3.2 b,3.7,...,3.8,3.3,3.5,4.0,3.5,3.4,3.9,4.3,4.4,4.0
2,PC_ACT,T,BG,:,:,:,:,9.6,12.6 b,12.0,...,4.1,2.9,3.0,4.7 b,6.3 b,6.8,7.4,6.9,5.6,4.5
3,PC_ACT,T,CY,:,:,:,:,:,:,:,...,0.7,0.5,0.6 b,1.3,1.6,3.6,6.1,7.7,6.8,5.8
4,PC_ACT,T,CZ,:,:,2.0,3.2,4.2,4.2,3.7,...,2.8,2.2,2.0,3.0,2.7 b,3.0,3.0,2.7,2.4,1.7


In [140]:
tmp2 = pd.read_csv(DATA_FOLDER + 'teilm022.tsv' , sep='\t|,|[|]', engine='python')
tmp2.head()
tmp3 = tmp2[['age','sex','unit','geo\\time','2017M09']].loc[range(0,5),:] # tmp2['sex']=='F' 
tmp3
#list(tmp3.loc[tmp3['geo\\time'] == 'BE', '2017M09'])[0]

,age,sex,unit,geo\time,2017M09
0,Y25-74,F,PC_ACT,AT,4.4
1,Y25-74,F,PC_ACT,BE,5.9
2,Y25-74,F,PC_ACT,BG,5.4
3,Y25-74,F,PC_ACT,CY,9.6
4,Y25-74,F,PC_ACT,CZ,3.2


In [138]:
def unemployment_color_test(id_):
    """TEST"""
    is_present = list(tmp3.loc[tmp3['geo\\time'] == id_, '2017M09'])
    if not is_present:
        return '#0000ff'
    else:
        return '#ff0000'

In [141]:
test_map = folium.Map([50,5], tiles='cartodbpositron', zoom_start=4)
folium.TopoJson(
    geo_json_data_europe,
    object_path='objects.europe',
    style_function=lambda feature: {
        'fillColor': unemployment_color_test(feature['id']),
        'color' : 'black',
        'weight' : 2,
        'dashArray' : '5, 5'
        }
    ).add_to(test_map)

test_map